## PNL Sepana Data exploration.

In [ ]:
!pip3 install polars

In [ ]:
!pip3 install pyarrow --force

In [ ]:
!pip3 install s3fs

In [9]:
import polars as pl
import pyarrow.parquet as pq
import s3fs

fs = s3fs.S3FileSystem()
path = ""

## TOC:
* [Read Data](#Read-Data)
* [Transactions](#Transactions)
* [Logs](#Logs)
* [Token Transfers](#Token-Transfers)


## Read Data

In [48]:

transactions_ds = pq.ParquetDataset(f"s3://{path}/transactions", filesystem=fs)
transactions_df = pl.from_arrow(transactions_ds.read())

In [49]:
transactions_df.schema

{'hash': polars.datatypes.Utf8,
 'nonce': polars.datatypes.Int64,
 'transaction_index': polars.datatypes.Int64,
 'from_address': polars.datatypes.Utf8,
 'to_address': polars.datatypes.Utf8,
 'value': polars.datatypes.Float64,
 'gas': polars.datatypes.Int64,
 'gas_price': polars.datatypes.Int64,
 'input': polars.datatypes.Utf8,
 'receipt_cumulative_gas_used': polars.datatypes.Int64,
 'receipt_gas_used': polars.datatypes.Int64,
 'receipt_contract_address': polars.datatypes.Utf8,
 'receipt_root': polars.datatypes.Utf8,
 'receipt_status': polars.datatypes.Int64,
 'block_number': polars.datatypes.Int64,
 'block_hash': polars.datatypes.Utf8,
 'max_fee_per_gas': polars.datatypes.Int64,
 'max_priority_fee_per_gas': polars.datatypes.Int64,
 'transaction_type': polars.datatypes.Int64,
 'receipt_effective_gas_price': polars.datatypes.Int64,
 'block_timestamp': datetime[ns],
 'function_name': polars.datatypes.Utf8,
 'contract_name': polars.datatypes.Utf8,
 'decoded_data': polars.datatypes.Utf8,
 '

In [53]:
logs_ds = pq.ParquetDataset(f"s3://{path}/logs", filesystem=fs)

logs_df = pl.from_arrow(logs_ds.read())

In [54]:
logs_df.schema

{'log_index': polars.datatypes.Int64,
 'transaction_hash': polars.datatypes.Utf8,
 'transaction_index': polars.datatypes.Int64,
 'address': polars.datatypes.Utf8,
 'data': polars.datatypes.Utf8,
 'topics': list[str],
 'block_number': polars.datatypes.Int64,
 'block_hash': polars.datatypes.Utf8,
 'block_timestamp': datetime[ns],
 'event_name': polars.datatypes.Utf8,
 'contract_name': polars.datatypes.Utf8,
 'decoded_data': polars.datatypes.Utf8,
 'schema': polars.datatypes.Utf8}

### Transactions

In [17]:
transactions_df.select(["block_number", "block_timestamp", "from_address", "to_address", "contract_name", "function_name", "decoded_data"])

block_number,block_timestamp,from_address,to_address,contract_name,function_name,decoded_data
i64,datetime[ns],str,str,str,str,str
16175194,2022-12-13 10:29:35,"""0xa939bffb33be...","""0x9f8f72aa9304...","""Maker: MKR Tok...","""transfer""","""{""dst"": ""0xA46..."
16168059,2022-12-12 10:33:59,"""0xc48ea4463cde...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0x912..."
16144161,2022-12-09 02:26:35,"""0x59529248afe6...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0x020..."
16168346,2022-12-12 11:31:59,"""0x237bc626b64d...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0xa60..."
16164582,2022-12-11 22:54:47,"""0x6f01c6ac94f2...","""0x9f8f72aa9304...","""Maker: MKR Tok...","""transfer""","""{""dst"": ""0x75e..."
16168853,2022-12-12 13:13:47,"""0x55d9bdb8f45b...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0x1f6..."
16177263,2022-12-13 17:27:59,"""0xd1557a4b7078...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0x9cB..."
16135196,2022-12-07 20:24:47,"""0x97f2afa152f4...","""0x6b175474e890...","""Maker: Dai Sta...","""approve""","""{""usr"": ""0xAd1..."
16111660,2022-12-04 13:12:11,"""0xeb0fd912e903...","""0x6b175474e890...","""Maker: Dai Sta...","""transfer""","""{""dst"": ""0xE70..."


In [66]:
transctions_json = transactions_df.select(["block_number", "hash", "block_timestamp", "from_address", "to_address", "contract_name", "function_name", "decoded_data", "schema"]).to_dicts()

In [67]:
## Decoded Data is string serialized dictionary so lets deserialize it.
import json
for txn in transctions_json:
    if txn["decoded_data"]:
        txn["decoded_data"] = json.loads(txn["decoded_data"])

In [68]:
len(transctions_json)

36247

In [69]:
transctions_json[:20]

[{'block_number': 16175194,
  'hash': '0x58a46b8a275274b83cbb27914aab9ca1eaa7b4878e202a4ec35eebe6d6a0e16b',
  'block_timestamp': datetime.datetime(2022, 12, 13, 10, 29, 35),
  'from_address': '0xa939bffb33bed18dc0063737cb9ea41a1fdb2fe3',
  'to_address': '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2',
  'contract_name': 'Maker: MKR Token',
  'function_name': 'transfer',
  'decoded_data': {'dst': '0xA46733E3658d3e04DdAf3BAcD4dbE65ea37437D3',
   'wad': 14800000000000000},
  'schema': '[{"name": "dst", "type": "address"}, {"name": "wad", "type": "uint256"}]'},
 {'block_number': 16168059,
  'hash': '0x57f76a705edb4b16573204047ad496064a293d29b2596831718a82fc2c11c902',
  'block_timestamp': datetime.datetime(2022, 12, 12, 10, 33, 59),
  'from_address': '0xc48ea4463cde37b6993c45842195969640e9d8c4',
  'to_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
  'contract_name': 'Maker: Dai Stablecoin',
  'function_name': 'transfer',
  'decoded_data': {'dst': '0x912fD21d7a69678227fE6d08C64222Db414

### Logs

In [34]:
logs_df

hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,block_timestamp,function_name,contract_name,decoded_data,schema
str,i64,i64,str,str,f64,i64,i64,str,i64,i64,str,str,i64,i64,str,i64,i64,i64,i64,datetime[ns],str,str,str,str
"""0x58a46b8a2752...",3724,60,"""0xa939bffb33be...","""0x9f8f72aa9304...",0.0,500000,15104426464,"""0xa9059cbb0000...",6434200,54326,null,null,1,16175194,"""0x8be112fe823b...",57430981425,2000000000,2,15104426464,2022-12-13 10:29:35,"""transfer""","""Maker: MKR Tok...","""{""dst"": ""0xA46...","""[{""name"": ""dst..."
"""0x57f76a705edb...",0,10,"""0xc48ea4463cde...","""0x6b175474e890...",0.0,34718,32432675571,"""0xa9059cbb0000...",735873,29918,null,null,1,16168059,"""0x44d70f85b44e...",null,null,0,32432675571,2022-12-12 10:33:59,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0x912...","""[{""internalTyp..."
"""0xc072f9d780fb...",119,63,"""0x59529248afe6...","""0x6b175474e890...",0.0,77763,13805712526,"""0xa9059cbb0000...",4703005,47042,null,null,1,16144161,"""0xb6a1e23416d5...",19716253530,1500000000,2,13805712526,2022-12-09 02:26:35,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0x020...","""[{""internalTyp..."
"""0x01286cc27990...",73,76,"""0x237bc626b64d...","""0x6b175474e890...",0.0,52077,14527055918,"""0xa9059cbb0000...",4953415,34718,null,null,1,16168346,"""0xc2d021188409...",20140000000,1550000000,2,14527055918,2022-12-12 11:31:59,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0xa60...","""[{""internalTyp..."
"""0x8a8612030d6b...",13644,1,"""0x6f01c6ac94f2...","""0x9f8f72aa9304...",0.0,55893,20000000000,"""0xa9059cbb0000...",190921,32462,null,null,1,16164582,"""0x928d3afeaec2...",null,null,0,20000000000,2022-12-11 22:54:47,"""transfer""","""Maker: MKR Tok...","""{""dst"": ""0x75e...","""[{""name"": ""dst..."
"""0x446fcc727869...",15,187,"""0x55d9bdb8f45b...","""0x6b175474e890...",0.0,51818,14342676588,"""0xa9059cbb0000...",13944865,47018,null,null,1,16168853,"""0xa557f17919d5...",27833000000,160000000,2,14342676588,2022-12-12 13:13:47,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0x1f6...","""[{""internalTyp..."
"""0xaed4d4d5cd0c...",64,144,"""0xd1557a4b7078...","""0x6b175474e890...",0.0,77727,30324290657,"""0xa9059cbb0000...",12483608,51818,null,null,1,16177263,"""0x03aae1040223...",38536731974,258594922,2,30324290657,2022-12-13 17:27:59,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0x9cB...","""[{""internalTyp..."
"""0xd6f7434753dc...",136,123,"""0x97f2afa152f4...","""0x6b175474e890...",0.0,51104,15541334558,"""0x095ea7b30000...",10855244,46458,null,null,1,16135196,"""0xe8b16f90a1f5...",20324435261,1272446115,2,15541334558,2022-12-07 20:24:47,"""approve""","""Maker: Dai Sta...","""{""usr"": ""0xAd1...","""[{""internalTyp..."
"""0xc8ee9cf75e8f...",1446,15,"""0xeb0fd912e903...","""0x6b175474e890...",0.0,2000000,18021170356,"""0xa9059cbb0000...",1448761,51818,null,null,1,16111660,"""0xd0c098b7bfdb...",null,null,0,18021170356,2022-12-04 13:12:11,"""transfer""","""Maker: Dai Sta...","""{""dst"": ""0xE70...","""[{""internalTyp..."


In [89]:
logs_df.groupby("event_name").count()

event_name,count
str,u32
"""""",41897
"""Registered""",13
"""Transfer""",133177
"""Burn""",2
"""Distributed""",18
"""Join""",3
"""Approval""",18373
"""Created""",174
"""Etch""",19


In [86]:
poke_events = logs_df.filter(pl.col("event_name") == "Poke").to_dicts()


In [88]:
poke_events[:5]

[{'log_index': 241,
  'transaction_hash': '0x3a362042879208fd5cf53eff03d49321a6696214085bf19f4eaef3fb8c697cff',
  'transaction_index': 93,
  'address': '0x65c79fcb50ca1594b025960e539ed7a9a6d434a3',
  'data': '0x554e495632555344434554482d410000000000000000000000000000000000000000000000000000000000000000000000000000007bf588807bbbd8fe485b660000000000000000000000000000000000180d1da538ee1709cd6307d1e61200',
  'topics': ['0xdfd7467e425a8107cfd368d159957692c25085aacbcf5228ce08f10f2146486e'],
  'block_number': 16104454,
  'block_hash': '0xcef3c1b234679e035844928717d592d1096678d7b8d9601d501c8a9ff83a0622',
  'block_timestamp': datetime.datetime(2022, 12, 3, 13, 1, 47),
  'event_name': 'Poke',
  'contract_name': 'Maker: MCD Spot',
  'decoded_data': '{"ilk": "0x554e495632555344434554482d41000000000000000000000000000000000000", "val": "0x0000000000000000000000000000000000000000007bf588807bbbd8fe485b66", "spot": 124881144680247063090755285000000000}',
  'schema': '[{"indexed": false, "internalType":

### Tracing a single transaction hash.

In [99]:
transactions_df.filter(pl.col("hash") == "0x0cfb8438816443efd6ed38588b75ac2ec7891270c2fb9887d5627d6dcc6e78e3").to_dicts()

[{'hash': '0x0cfb8438816443efd6ed38588b75ac2ec7891270c2fb9887d5627d6dcc6e78e3',
  'nonce': 578,
  'transaction_index': 80,
  'from_address': '0xd4203e6ffc798765dea121d795b2a56684c1c738',
  'to_address': '0xc4e0f3ec24972c75df7c716922096f4270b7bb4e',
  'value': 0.0,
  'gas': 46205,
  'gas_price': 11431557383,
  'input': '0xa22cb465000000000000000000000000f42aa99f011a1fa7cda90e5e98b277e306bca83e0000000000000000000000000000000000000000000000000000000000000001',
  'receipt_cumulative_gas_used': 5977609,
  'receipt_gas_used': 46205,
  'receipt_contract_address': None,
  'receipt_root': None,
  'receipt_status': 1,
  'block_number': 16116740,
  'block_hash': '0x108b08df72ad9f8248d712ca6d93605f861696e0d8013337f5aa54e22458fbff',
  'max_fee_per_gas': 16264383033,
  'max_priority_fee_per_gas': 1500000000,
  'transaction_type': 2,
  'receipt_effective_gas_price': 11431557383,
  'block_timestamp': datetime.datetime(2022, 12, 5, 6, 13, 59),
  'function_name': 'setApprovalForAll',
  'contract_name': 

In [100]:
logs_df.filter(pl.col("transaction_hash") == "0x0cfb8438816443efd6ed38588b75ac2ec7891270c2fb9887d5627d6dcc6e78e3").to_dicts()

[{'log_index': 116,
  'transaction_hash': '0x0cfb8438816443efd6ed38588b75ac2ec7891270c2fb9887d5627d6dcc6e78e3',
  'transaction_index': 80,
  'address': '0xc4e0f3ec24972c75df7c716922096f4270b7bb4e',
  'data': '0x0000000000000000000000000000000000000000000000000000000000000001',
  'topics': ['0x17307eab39ab6107e8899845ad3d59bd9653f200f220920489ca2b5937696c31',
   '0x000000000000000000000000d4203e6ffc798765dea121d795b2a56684c1c738',
   '0x000000000000000000000000f42aa99f011a1fa7cda90e5e98b277e306bca83e'],
  'block_number': 16116740,
  'block_hash': '0x108b08df72ad9f8248d712ca6d93605f861696e0d8013337f5aa54e22458fbff',
  'block_timestamp': datetime.datetime(2022, 12, 5, 6, 13, 59),
  'event_name': 'ApprovalForAll',
  'contract_name': 'Mirror: CrowdfundWithPodiumEditions',
  'decoded_data': '{"owner": "0xD4203e6fFC798765Dea121D795b2a56684c1c738", "operator": "0xf42aa99F011A1fA7CDA90E5E98b277E306BcA83e", "approved": true}',
  'schema': '[{"indexed": true, "internalType": "address", "name": "o

### Token Transfers

In [3]:

token_transfers_ds = pq.ParquetDataset(f"s3://{path}/token_transfers", filesystem=fs)
token_transfers_df = pl.from_arrow(token_transfers_ds.read())

In [7]:
token_transfers_df

token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash,symbol,name,decimals
str,str,str,str,str,i64,datetime[ns],i64,str,str,str,str
"""0x6b175474e890...","""0xacece676acdb...","""0xa9d1e08c7793...","""80000000000000...","""0x73620433bc6d...",43,2022-12-15 01:21:11,16186778,"""0x8a5329d361fe...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0xb95bd0793bcc...","""0x2d6bb700ed0f...","""20174907811563...","""0x5b3031116d1d...",312,2022-12-15 01:19:11,16186768,"""0xcf2144720416...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0x94cc38e2be86...","""0x74de5d4fcbf6...","""14866932684187...","""0x9385ab671879...",70,2022-12-07 06:37:47,16131116,"""0xa3819ee9e1df...","""DAI""","""Dai Stablecoin...","""18"""
"""0x9f8f72aa9304...","""0x28c6c06298d5...","""0xbb26df665c59...","""11275200000000...","""0xf5af667d3f41...",56,2022-12-07 07:52:11,16131476,"""0x2df2a7cd4b37...","""MKR           ...","""Maker         ...","""18"""
"""0x6b175474e890...","""0xb2a2a6a69e7a...","""0xdca17eedc1aa...","""96164383500000...","""0xce4e9e424192...",130,2022-12-07 14:50:47,16133534,"""0x6fa0f20da2a5...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0x7c61134c8146...","""0x777777c9898d...","""70000000000000...","""0x26edce6e5efa...",250,2022-12-07 20:22:35,16135185,"""0xb9dc9bb4fe3d...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0xff1f2b4adb9d...","""0x4cf32670a536...","""28600000000000...","""0xdad9d3091f13...",165,2022-12-06 04:16:59,16123307,"""0x3e6266e1f347...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0x000000000000...","""0xe0b8ea01093c...","""30000000000000...","""0xd9453ae6a5fe...",46,2022-12-06 18:53:59,16127671,"""0x31e7138ea741...","""DAI""","""Dai Stablecoin...","""18"""
"""0x6b175474e890...","""0x2a1530c4c41d...","""0x60594a405d53...","""68063186817915...","""0x2e105456f1b1...",65,2022-12-07 07:11:23,16131279,"""0xc93d9b74b582...","""DAI""","""Dai Stablecoin...","""18"""


In [8]:
token_transfers_df.to_dicts()[:5]

[{'token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
  'from_address': '0xacece676acdb01d7996d9e2ea65d46d29d250e51',
  'to_address': '0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43',
  'value': '8000000000000000000000',
  'transaction_hash': '0x73620433bc6d61a268764260781806bf7fe6fd1a3244663e1d53a30dc9d3db6f',
  'log_index': 43,
  'block_timestamp': datetime.datetime(2022, 12, 15, 1, 21, 11),
  'block_number': 16186778,
  'block_hash': '0x8a5329d361fe9a01a05dde37d7da4f6791c2cfe42858b3e1dad38f9832580052',
  'symbol': 'DAI',
  'name': 'Dai Stablecoin',
  'decimals': '18'},
 {'token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
  'from_address': '0xb95bd0793bcc5524af358ffaae3e38c3903c7626',
  'to_address': '0x2d6bb700ed0fc6d1fcb9057c5c12edb0d1c06f4b',
  'value': '201749078115639224560',
  'transaction_hash': '0x5b3031116d1d58749e91b6480f5748397ced1cd444efb3acae5f8915c35cb28c',
  'log_index': 312,
  'block_timestamp': datetime.datetime(2022, 12, 15, 1, 19, 11),
  'block_n